# Activity 1: Computing Optimal Portfolios with Form Closed Solution
----
In this activity we are going to analyze properties of the optimal portfolio within the context of the two asset example we explore in class. 

The optimal solution in that case is given by:
> $x_1 = \frac{R^{e,2}\sigma_{12}-R^{e,1}\sigma^2_2}
{\gamma(\sigma_{12}^2-\sigma_1^2 \sigma_2^2)}$,
 $x_2 = \frac{R^{e,1}\sigma_{12}-R^{e,2}\sigma^2_1}
{\gamma(\sigma_{12}^2-\sigma_1^2 \sigma_2^2)}$

We are going to define a function to handle the computation of these values and the expected return and volatility of a portfolio. 

Note: expected values and volatilities are computed using the risky components of the portfolio (assets 1 and 2)

In [58]:
import numpy as np

def optWeights(r,s,cr, g):
    #This function computes optimal weights using closed-form solution
    r_1 = r[0]
    r_2 = r[1]
    sigma_1 = s[0]
    sigma_2 = s[1]
    cov = cr*sigma_1*sigma_2
    x = np.zeros(3)
    x[0] = (r_2*cov - r_1*sigma_2**2)/(g*(cov**2 -(sigma_1**2)*(sigma_2**2)))
    x[1] = (r_1*cov - r_2*sigma_1**2)/(g*(cov**2 -(sigma_1**2)*(sigma_2**2)))
    x[2] = 1 - x[0] - x[1]
    return x

In [59]:
ret = np.array([0.09,0.15])
sigma = np.array([0.20,0.25])
corr = 0.5
g = 3
x = optWeights(ret,sigma,corr,g)
print(x)

[0.33333333 0.66666667 0.        ]


In [71]:
def expRetVar(ret,sigma,corr,g,weights):
    #This function computes the expected return and volatility of portfolio
    X = weights[0:2]
    expRet = np.dot(X.T,ret) #This is equivalent to: np.sum(X*ret)
    SigMtx = [[sigma[0]**2,corr*sigma[0]*sigma[1]],\
              [corr*sigma[0]*sigma[1],sigma[1]**2]]
    expVar = np.sqrt(np.dot(X.T,np.dot(SigMtx,X)))
    return expRet, expVar


In [83]:
exR, exV = expRetVar(ret,sigma,corr,g,x)
print("The expected value of the portfolio is %5.4f and the volatility is %5.4f"%(exR,exV))
print("The function value is %4.7f"%(exR-(g/2)*(exV**2)))

The expected value of the portfolio is 0.1300 and the volatility is 0.2082
The function value is 0.0650000


Let's look at the impact of the aversion coefficient $\gamma$ on the optimal weights, expected return, and expected volatility:

In [49]:
corr = 0.5
gamma = range(1,9,1)
num_elem = len(gamma)
x = np.zeros((num_elem,3))
exRet = np.zeros(num_elem)
exVol = np.zeros(num_elem)
for i in range(num_elem):
    x[i,:] = optWeights(ret,sigma,corr,gamma[i])
    exRet[i], exVol[i] = expRetVar(ret,sigma,corr,gamma[i],x[i,:])
    print("For gamma=%5.2f, the optimal weights are: (%5.2f,%5.2f,%5.2f)."%(gamma[i],x[i,0],x[i,1],x[i,2]))
    print("The expected return and volatility are %5.2f and %5.2f, respectively." %(exRet[i],exVol[i]))



For gamma= 1.00, the optimal weights are: ( 1.00, 2.00,-2.00).
The expected return and volatility are  0.39 and  0.62, respectively.
For gamma= 2.00, the optimal weights are: ( 0.50, 1.00,-0.50).
The expected return and volatility are  0.20 and  0.31, respectively.
For gamma= 3.00, the optimal weights are: ( 0.33, 0.67, 0.00).
The expected return and volatility are  0.13 and  0.21, respectively.
For gamma= 4.00, the optimal weights are: ( 0.25, 0.50, 0.25).
The expected return and volatility are  0.10 and  0.16, respectively.
For gamma= 5.00, the optimal weights are: ( 0.20, 0.40, 0.40).
The expected return and volatility are  0.08 and  0.12, respectively.
For gamma= 6.00, the optimal weights are: ( 0.17, 0.33, 0.50).
The expected return and volatility are  0.07 and  0.10, respectively.
For gamma= 7.00, the optimal weights are: ( 0.14, 0.29, 0.57).
The expected return and volatility are  0.06 and  0.09, respectively.
For gamma= 8.00, the optimal weights are: ( 0.12, 0.25, 0.62).
The ex

# Activity 2: Compute Optimal Portfolio Using Numerical Optimization
----

We are going to solve the optimization problem:

> $\max_{x} x'R^{e} - \frac{\gamma}{2} x'\Omega x$

which is equivalent to solving:
> $\min_{x} -(x'R^{e} - \frac{\gamma}{2} x'\Omega x)$

To use numerical optimization, we have to first define the function that we want to minimize

In [1]:
def min_func_meanVar(X,ret,Omega,gamma):
    #This function computes the negative of the objective function
    expRet = np.dot(X.T,ret) #This is equivalent to: np.sum(X*ret)
    expVar = np.dot(X.T,np.dot(Omega,X))
    val_funct = expRet - (gamma/2)*expVar
    return -1*val_funct

To carry out the optimization, we need to follow the next instructions

In [4]:
import scipy.optimize as sco #module where optimization functions are available
import numpy as np

g = 3
ret = np.array([0.09,0.15])
sigma = np.array([0.20,0.25])
corr = 0.5
cov = corr*sigma[0]*sigma[1]
Omega = np.array([[sigma[0]**2,cov],[cov,sigma[1]**2]])
#Set up the optimization problem
args = (ret, Omega,g)
num_assets = len(ret)
init_x = np.ones(num_assets)*(1/num_assets) #set equal weights as starting point
result = sco.minimize(min_func_meanVar, init_x, args=args, method='SLSQP')
print(result)
opt_x = result.x
print("The optimal solution is")
print(opt_x)

     fun: -0.06499998122696451
     jac: array([-3.91509384e-05,  3.45520675e-05])
 message: 'Optimization terminated successfully.'
    nfev: 32
     nit: 8
    njev: 8
  status: 0
 success: True
       x: array([0.33274474, 0.66708637])
The optimal solution is
[0.33274474 0.66708637]
